In [8]:
import osparc
import time
from pathlib import Path
from osparc.api import FilesApi, SolversApi
from osparc.models import File, Job, JobInputs, JobOutputs, JobStatus, Solver
import os
print(osparc.__version__)

OSPARC_API_KEY = os.environ['OSPARC_API_KEY']
OSPARC_API_SECRET = os.environ['OSPARC_API_SECRET']

cfg = osparc.Configuration(
    username=OSPARC_API_KEY,
    password=OSPARC_API_SECRET,
)
print(cfg.host)

with osparc.ApiClient(cfg) as api_client:

    files_api = FilesApi(api_client)
    current_folder = os.getcwd()
    input_file_path = os.path.join(current_folder, 'tmp', 'fake-uuid-for-sample-data', 'INPUT_FOLDER', 'input.json')
    input_file1: File = files_api.upload_file(file="main.zip")
    input_file2: File = files_api.upload_file(file=input_file_path)

    solvers_api = SolversApi(api_client)
    solver: Solver = solvers_api.get_solver_release(
        "simcore/services/comp/osparc-python-runner", "1.2.0"
    )

    job: Job = solvers_api.create_job(
        solver.id,
        solver.version,
        JobInputs(
            {
                "input_2": input_file2,
                "input_1": input_file1,
            }
        ),
    )

    status: JobStatus = solvers_api.start_job(solver.id, solver.version, job.id)
    while not status.stopped_at:
        time.sleep(3)
        status = solvers_api.inspect_job(solver.id, solver.version, job.id)
        print("Solver progress", f"{status.progress}/100", flush=True)

    outputs: JobOutputs = solvers_api.get_job_outputs(solver.id, solver.version, job.id)

    print(f"Job {outputs.job_id} got these results:")
    for output_name, result in outputs.results.items():
        print(output_name, "=", result)

    results_file: File = outputs.results["output_1"]
    download_path: str = files_api.download_file(file_id=results_file.id)
    print(download_path)

0.4.3
https://api.osparc.io


ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Content-Length': '123', 'Content-Type': 'application/json', 'Date': 'Thu, 22 Jul 2021 06:59:47 GMT', 'Server': 'uvicorn', 'Vary': 'Accept-Encoding, Accept-Encoding'})
HTTP response body: {"errors":["1 validation error for PresignedLink\n__root__\n  PresignedLink expected dict not NoneType (type=type_error)"]}


In [9]:
import osparc
import time
from pathlib import Path
from osparc.api import FilesApi, SolversApi
from osparc.models import File, Job, JobInputs, JobOutputs, JobStatus, Solver
import os
print(osparc.__version__)
from zipfile import ZipFile

cfg = osparc.Configuration(
    username=os.environ["OSPARC_API_KEY"],
    password=os.environ["OSPARC_API_SECRET"],
)
print(cfg.host)

with osparc.ApiClient(cfg) as api_client:

    files_api = FilesApi(api_client)
    current_folder = os.getcwd()
    input_file_path = os.path.join(current_folder, 'tmp', 'fake-uuid-for-sample-data', 'INPUT_FOLDER', 'matlab-input-folder.zip')
    input_file1: File = files_api.upload_file(file=input_file_path)

    solvers_api = SolversApi(api_client)
    solver: Solver = solvers_api.get_solver_release(
        "simcore/services/comp/mat-parser", "1.0.0"
    )

    job: Job = solvers_api.create_job(
        solver.id,
        solver.version,
        JobInputs(
            {
                "input_1": input_file1,
            }
        ),
    )

    status: JobStatus = solvers_api.start_job(solver.id, solver.version, job.id)
    while not status.stopped_at:
        time.sleep(3)
        status = solvers_api.inspect_job(solver.id, solver.version, job.id)
        print("Solver progress", f"{status.progress}/100", flush=True)

    outputs: JobOutputs = solvers_api.get_job_outputs(solver.id, solver.version, job.id)

    print(f"Job {outputs.job_id} got these results:")
    for output_name, result in outputs.results.items():
        print(output_name, "=", result)

    results_file: File = outputs.results["output_1"]
    download_path: str = files_api.download_file(file_id=results_file.id)
    print(download_path)
    if os.path.isfile(download_path) or os.path.isfir(download_path):
        print("Unzipping")
        unzip_location = os.path.join(current_folder, 'tmp', 'fake-uuid-for-sample-data', 'OUTPUT_FOLDER')
        with ZipFile(download_path, 'r') as zipObj:
            zipObj.extractall(unzip_location)

0.4.3
https://api.osparc.io


ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Length': '38', 'Content-Type': 'application/json', 'Date': 'Thu, 22 Jul 2021 07:00:09 GMT', 'Server': 'uvicorn', 'Vary': 'Accept-Encoding, Accept-Encoding'})
HTTP response body: {"errors":["Invalid API credentials"]}


Unzipping
